### Задание 1

In [1]:
"""
Необходимо создать 3 класса и взаимосвязь между ними (Student, Teacher,
Homework)
Наследование в этой задаче использовать не нужно.
Для работы с временем использовать модуль datetime

1. Homework принимает на вход 2 атрибута: текст задания и количество дней
на это задание
Атрибуты:
    text - текст задания
    deadline - хранит объект datetime.timedelta с количеством
    дней на выполнение
    created - c точной датой и временем создания
Методы:
    is_active - проверяет не истекло ли время на выполнение задания,
    возвращает boolean

2. Student
Атрибуты:
    last_name
    first_name
Методы:
    do_homework - принимает объект Homework и возвращает его же,
    если задание уже просрочено, то печатет 'You are late' и возвращает None

3. Teacher
Атрибуты:
     last_name
     first_name
Методы:
    create_homework - текст задания и количество дней на это задание,
    возвращает экземпляр Homework
    Обратите внимание, что для работы этого метода не требуется сам объект.

PEP8 соблюдать строго.
Всем перечисленным выше атрибутам и методам классов сохранить названия.
К названием остальных переменных, классов и тд. подходить ответственно -
давать логичные подходящие имена. """


import datetime

class Homework:
    """класс Домашняя работа  
    text - текст задания,
    deadline - хранит объект datetime.timedelta с количеством,
    дней на выполнение,
    created - время создания"""
    
    
    
    def __init__(self, text, days):
        """свойства домашней работы"""
        self.text = text
        self.deadline = datetime.timedelta(days=days)
        self.created = datetime.datetime.now()
        
    
    def is_active(self):
        """проверяет, не истекло ли время"""
        return datetime.datetime.now() < self.created + self.deadline


class Student:
    """класс Студент"""
    def __init__(self, last_name, first_name):
        """признаки студента"""
        self.last_name = last_name
        self.first_name = first_name

    def do_homework(self, homework):
        """принимает объект Homework и возвращает его, если задание не просрочено или выводит "You are late" """
        if homework.is_active():
            return homework
        else:
            print("You are late")
            return None


class Teacher:
    """класс Учитель"""
    def __init__(self, last_name, first_name):
        """признаки учителя"""
        self.last_name = last_name
        self.first_name = first_name

    def create_homework(self, text, days):
        """создает экземпляр Homework"""
        return Homework(text, days)


# Пример использования классов
if __name__ == '__main__':
    teacher = Teacher('Daniil', 'Shadrin')
    student = Student('Roman', 'Petrov')

    expired_homework = teacher.create_homework('Learn functions', 0)
    print(expired_homework.created)  # Время создания задания
    print(expired_homework.deadline)  # 0:00:00
    print(expired_homework.text)  # 'Learn functions'

    # Создание задания с использованием функции
    create_homework_too = teacher.create_homework
    oop_homework = create_homework_too('create 2 simple classes', 5)
    print(oop_homework.deadline)  # 5 days, 0:00:00

    # Выполнение заданий
    student.do_homework(oop_homework)  # Выполнение задания
    student.do_homework(expired_homework)  # Вывод: You are late

2025-03-10 23:42:43.615324
0:00:00
Learn functions
5 days, 0:00:00
You are late


### Задание 2

In [2]:
"""
В этом задании будем улучшать нашу систему классов из задания прошлой лекции
(Student, Teacher, Homework)
Советую обратить внимание на defaultdict из модуля collection для
использования как общую переменную


1. Как то не правильно, что после do_homework мы возвращаем все тот же
объект - будем возвращать какой-то результат работы (HomeworkResult)

HomeworkResult принимает объект автора задания, принимает исходное задание
и его решение в виде строки
Атрибуты:
    homework - для объекта Homework, если передан не этот класс -  выкинуть
    подходящие по смыслу исключение с сообщением:
    'You gave a not Homework object'

    solution - хранит решение ДЗ как строку
    author - хранит объект Student
    created - c точной датой и временем создания

2. Если задание уже просрочено хотелось бы видеть исключение при do_homework,
а не просто принт 'You are late'.
Поднимайте исключение DeadlineError с сообщением 'You are late' вместо print.

3. Student и Teacher имеют одинаковые по смыслу атрибуты
(last_name, first_name) - избавиться от дублирования с помощью наследования

4.
Teacher
Атрибут:
    homework_done - структура с интерфейсом как в словаря, сюда поподают все
    HomeworkResult после успешного прохождения check_homework
    (нужно гаранитровать остутствие повторяющихся результатов по каждому
    заданию), группировать по экземплярам Homework.
    Общий для всех учителей. Вариант ипользования смотри в блоке if __main__...
Методы:
    check_homework - принимает экземпляр HomeworkResult и возвращает True если
    ответ студента больше 5 символов, так же при успешной проверке добавить в
    homework_done.
    Если меньше 5 символов - никуда не добавлять и вернуть False.

    reset_results - если передать экземпряр Homework - удаляет только
    результаты этого задания из homework_done, если ничего не передавать,
    то полностью обнулит homework_done.

PEP8 соблюдать строго.
Всем перечисленным выше атрибутам и методам классов сохранить названия.
К названием остальных переменных, классов и тд. подходить ответственно -
давать логичные подходящие имена.
"""
import datetime
from collections import defaultdict

    
class DeadlineError(Exception):
    """исключение для обработки просроченных заданий"""
    pass


class Homework:
    """класс Домашняя работа  
    text - текст задания,
    deadline - хранит объект datetime.timedelta с количеством,
    дней на выполнение,
    created - время создания"""    
    
    def __init__(self, text: str, days: int):
        """свойства домашней работы"""
        self.text = text
        self.deadline = datetime.timedelta(days=days)
        self.created = datetime.datetime.now()        
    
    def is_active(self) -> bool:
        """проверяет, не истекло ли время"""
        return datetime.datetime.now() < self.created + self.deadline

    
class HomeworkResult:
    """класс результат домашней работы
    homework - для объекта Homework, если передан не этот класс -  выкинуть
    подходящие по смыслу исключение с сообщением: 'You gave a not Homework object'
    solution - хранит решение ДЗ как строку
    author - хранит объект Student
    created - время создания"""
    def __init__(self, author, homework: Homework, solution: str):
        """свойства результата домашней работы"""
        if not isinstance(homework, Homework):#isinstance - функция для проверки, является ли объект экземпляром опред.класса
            raise TypeError('You gave a not Homework object')
        self.author = author
        self.homework = homework
        self.solution = solution
        self.created = datetime.datetime.now()
        
    def __repr__(self):
        return f"HomeworkResult(author={self.author.first_name} {self.author.last_name}, solution='{self.solution}')"

    
class Person:
    """суперкласс Человек"""
    def __init__(self, first_name: str, last_name: str):
        self.first_name = first_name
        self.last_name = last_name


class Student(Person):
    """класс Студент"""
    def do_homework(self, homework: Homework, solution: str) -> HomeworkResult:
        if not homework.is_active():
            raise DeadlineError('You are late')
        return HomeworkResult(self, homework, solution)

    
class Teacher(Person):
    """класс Учитель"""
    homework_done = defaultdict(list)  #хранение результатов
    
    def create_homework(self, text: str, days: int) -> Homework:
        """создает экземпляр Homework"""
        return Homework(text, days)
    
    def check_homework(self, homework_result: HomeworkResult) -> bool:
        """проверяет решение домашнего задания"""
        if any(result.author == homework_result.author for result in self.homework_done[homework_result.homework]):
            return False  #результат уже существует, не добавляем
        
        if len(homework_result.solution) > 5:
            self.homework_done[homework_result.homework].append(homework_result)
            return True
        return False
    
    @classmethod
    def reset_results(cls, homework=None):
        """сбрасывает результаты домашних заданий"""
        if homework:
            cls.homework_done.pop(homework, None)  #удаляет только результаты этого задания
        else:
            cls.homework_done.clear()  #полностью обнуляет

                
                
if __name__ == '__main__':
    opp_teacher = Teacher('Daniil', 'Shadrin')
    advanced_python_teacher = Teacher('Aleksandr', 'Smetanin')

    lazy_student = Student('Roman', 'Petrov')
    good_student = Student('Lev', 'Sokolov')

    oop_hw = opp_teacher.create_homework('Learn OOP', 1)
    docs_hw = opp_teacher.create_homework('Read docs', 5)

    result_1 = good_student.do_homework(oop_hw, 'I have done this hw')
    result_2 = good_student.do_homework(docs_hw, 'I have done this hw too')
    result_3 = lazy_student.do_homework(docs_hw, 'done')

    try:
        result_4 = HomeworkResult(good_student, "fff", "Solution")
    except Exception:
        print('There was an exception here')

    opp_teacher.check_homework(result_1)
    temp_1 = opp_teacher.homework_done

    advanced_python_teacher.check_homework(result_1)
    temp_2 = Teacher.homework_done
    assert temp_1 == temp_2

    opp_teacher.check_homework(result_2)
    opp_teacher.check_homework(result_3)

    print(Teacher.homework_done[oop_hw])
    Teacher.reset_results()


There was an exception here
[HomeworkResult(author=Lev Sokolov, solution='I have done this hw')]


### Задание 3

In [3]:
"""
Given a Tic-Tac-Toe 3x3 board (can be unfinished).
Write a function that checks if the are some winners.
If there is "x" winner, function should return "x wins!"
If there is "o" winner, function should return "o wins!"
If there is a draw, function should return "draw!"
If board is unfinished, function should return "unfinished!"

Example:
    [[-, -, o],
     [-, x, o],
     [x, o, x]]
    Return value should be "unfinished"

    [[-, -, o],
     [-, o, o],
     [x, x, x]]

     Return value should be "x wins!"

"""
from typing import List

def tic_tac_toe_checker(board: List[List[str]]) -> str:
    #проверка строк
    for row in board:
        if row[0] == row[1] == row[2] and row[0] != '-':
            return f"{row[0]} wins!"

    #проверка столбцов
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != '-':
            return f"{board[0][col]} wins!"

    #проверка диагоналей
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != '-':
        return f"{board[0][0]} wins!"
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != '-':
        return f"{board[0][2]} wins!"

    #проверка на ничью или незавершенность
    for row in board:
        if '-' in row:
            return "unfinished"
    
    return "draw!"






#примеры
print(tic_tac_toe_checker([['-', '-', 'o'],
                            ['-', 'x', 'o'],
                            ['x', 'o', 'x']]))  # Вывод: "unfinished"

print(tic_tac_toe_checker([['-', '-', 'o'],
                            ['-', 'o', 'o'],
                            ['x', 'x', 'x']]))  # Вывод: "x wins!"

unfinished
x wins!
